**Brief description**
---------------------

The Dataset is an anonymized sample of over 3,000,000 grocery orders from more than 200,000 Instacart users. 
The goal of a competition is to predict which previously purchased products will be in a user’s next order.
In this kernel, we will try to explore the data. Try to understand relationship between various features of the data before we move on to prediction model.

Loading Modules:
----------------

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt # plotting
import seaborn as sns # plotting

Loading Datsets:
----------------

In [ ]:
aisles = pd.read_csv('../input/aisles.csv', engine='c')
departments = pd.read_csv('../input/departments.csv', engine='c')
products = pd.read_csv('../input/products.csv', engine='c')
#Making one dataframe for commodities
df_com = pd.merge(left=pd.merge(left=products, right=aisles, how='left'), right=departments, how='left')

df_train = pd.read_csv('../input/order_products__train.csv', engine='c')
df_prior = pd.read_csv('../input/order_products__prior.csv', engine='c')
orders = pd.read_csv('../input/orders.csv', engine='c')

In [ ]:
del aisles
df_com.head()

In [ ]:
orders.head()

Consolidating Train Data with Orders and Commodity DataFrame
------------------------------------------------------------

In [ ]:
df_train = pd.merge(left = (pd.merge(left = df_train, right = orders, how = 'left', on = 'order_id')), right = df_com[['product_id', 'aisle_id', 'department_id']], how = 'left', on = 'product_id')
del orders
df_train['reordered'] = df_train.add_to_cart_order*df_train.reordered
df_train.head()

How are various Aisles distributed amongst various Departments?
---------------------------------------------------------------

In [ ]:
f, axarr = plt.subplots(7, 3, figsize=(12, 30))
for i,e in enumerate(departments.department.sort_values(ascending=True)):
    axarr[i//3, i%3].set_title('Dep: {}'.format(e))
    df_com[df_com.department==e].groupby(['aisle']).count()['product_id']\
    .sort_values(ascending=False).plot(kind='bar', color="#4CB391", ax=axarr[i//3, i%3])
f.subplots_adjust(hspace=2)

Relationship between Products[Top-50] Ordering and Re-ordering:
----------------------------------------

In [ ]:
aggregations = {'add_to_cart_order':'sum', 'reordered':'sum'}
prod_reorder = df_train.groupby('product_id').agg(aggregations)
prod_reorder = prod_reorder.sort_values('add_to_cart_order', ascending=False)[:50]

prod_user = df_train.groupby('user_id').agg(aggregations)
prod_user = prod_user.sort_values('add_to_cart_order', ascending=False)[:50]

In [ ]:
sns.set(style="white")

# Set up the figure
f, ax = plt.subplots(figsize=(10, 6))

# Draw the two density plots
ax = sns.kdeplot(prod_reorder.add_to_cart_order, prod_reorder.reordered,
                 cmap="Reds", shade=True, shade_lowest=False)

Relationship between Users[Top-50] Ordering and Re-ordering:
----------------------------------------

In [ ]:
sns.set(style="white")

# Set up the figure
f, ax = plt.subplots(figsize=(10, 6))

# Draw the two density plots
ax = sns.kdeplot(prod_user.add_to_cart_order, prod_user.reordered,
                 cmap="Blues", shade=True, shade_lowest=False)

Relationship between re-ordering and days_since_prior_order:
------------------------------------------------------------

*Due to so many data points, we are taking top 8000 points to understand the distribution.*

In [ ]:
from scipy.stats import kendalltau

df_train1 = df_train.sort_values('reordered', ascending=False)[:8000]

sns.set(style="ticks")

x = df_train1.reordered
y = df_train1.days_since_prior_order

sns.jointplot(x, y, kind="hex", stat_func=kendalltau, color="#4CB391")

Takeaways:
----------

 - Various aisles show Chi Square distribution when studied department-wise
 - Most of the re-ordered products are fixed with few outliers and little variance in re-ordering
 - KDE plot of user-wise[Top-50] re-ordering shows that the plot is less flat (as compared to product-wise chart). Chart shows high variance, which indicates that there is a scope of user retention and acquisition 

 